In [2]:
import pickle
import pathlib

import numpy as np
import pandas as pd

DATA_DIR = pathlib.Path.cwd().parent / 'data'
print(DATA_DIR)

c:\Users\Alan Matheus\Desktop\ayres\ames-1\data


In [3]:
clean_data_path = DATA_DIR / 'processed' / 'ames_minha1.pkl'

In [4]:
with open('../data/processed/ames_minha1.pkl', 'rb') as file:
    data = pickle.load(file)

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2724 entries, 0 to 2929
Data columns (total 80 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   MS.SubClass      2724 non-null   category
 1   MS.Zoning        2724 non-null   category
 2   Lot.Frontage     2724 non-null   float64 
 3   Lot.Area         2724 non-null   float64 
 4   Lot.Shape        2724 non-null   category
 5   Land.Contour     2724 non-null   category
 6   Lot.Config       2724 non-null   category
 7   Land.Slope       2724 non-null   category
 8   Neighborhood     2724 non-null   category
 9   Bldg.Type        2724 non-null   category
 10  House.Style      2724 non-null   category
 11  Overall.Qual     2724 non-null   category
 12  Overall.Cond     2724 non-null   category
 13  Year.Built       2724 non-null   float64 
 14  Roof.Style       2724 non-null   category
 15  Mas.Vnr.Type     2724 non-null   category
 16  Mas.Vnr.Area     2724 non-null   float64 
 17  

In [5]:
model_data = data.copy()

In [6]:
categorical_columns = []
ordinal_columns = []
for col in model_data.select_dtypes('category').columns:
    if model_data[col].cat.ordered:
        ordinal_columns.append(col)
    else:
        categorical_columns.append(col)

In [7]:
for col in ordinal_columns:
    codes, _ = pd.factorize(data[col], sort=True)
    model_data[col] = codes

In [8]:
original_data = model_data['Exterior']
encoded_data = pd.get_dummies(original_data)

aux_dataframe = encoded_data
aux_dataframe['Exterior'] = original_data.copy()

aux_dataframe.head().transpose()

,0,1,2,3,4
AsbShng,False,False,False,False,False
BrkFace,True,False,False,True,False
CemntBd,False,False,False,False,False
HdBoard,False,False,False,False,False
MetalSd,False,False,False,False,False
Plywood,False,False,False,False,False
Stucco,False,False,False,False,False
VinylSd,False,True,False,False,True
Wd Sdng,False,False,True,False,False
WdShing,False,False,False,False,False


In [9]:
original_data = model_data['Exterior']
encoded_data = pd.get_dummies(original_data, drop_first=True)

aux_dataframe = encoded_data
aux_dataframe['Exterior'] = original_data.copy()

aux_dataframe.head().transpose()

,0,1,2,3,4
BrkFace,True,False,False,True,False
CemntBd,False,False,False,False,False
HdBoard,False,False,False,False,False
MetalSd,False,False,False,False,False
Plywood,False,False,False,False,False
Stucco,False,False,False,False,False
VinylSd,False,True,False,False,True
Wd Sdng,False,False,True,False,False
WdShing,False,False,False,False,False
Other,False,False,False,False,False


In [10]:
model_data = pd.get_dummies(model_data, drop_first=True)

In [11]:
for cat in categorical_columns:
    dummies = []
    for col in model_data.columns:
        if col.startswith(cat + "_"):
            dummies.append(f'"{col}"')
    dummies_str = ', '.join(dummies)
    print(f'From column "{cat}" we made {dummies_str}\n')

From column "MS.SubClass" we made "MS.SubClass_30", "MS.SubClass_50", "MS.SubClass_60", "MS.SubClass_70", "MS.SubClass_80", "MS.SubClass_85", "MS.SubClass_90", "MS.SubClass_120", "MS.SubClass_160", "MS.SubClass_190", "MS.SubClass_Other"

From column "MS.Zoning" we made "MS.Zoning_RH", "MS.Zoning_RL", "MS.Zoning_RM"

From column "Land.Contour" we made "Land.Contour_HLS", "Land.Contour_Low", "Land.Contour_Lvl"

From column "Lot.Config" we made "Lot.Config_CulDSac", "Lot.Config_FR2", "Lot.Config_FR3", "Lot.Config_Inside"

From column "Neighborhood" we made "Neighborhood_BrDale", "Neighborhood_BrkSide", "Neighborhood_ClearCr", "Neighborhood_CollgCr", "Neighborhood_Crawfor", "Neighborhood_Edwards", "Neighborhood_Gilbert", "Neighborhood_IDOTRR", "Neighborhood_MeadowV", "Neighborhood_Mitchel", "Neighborhood_NAmes", "Neighborhood_NPkVill", "Neighborhood_NWAmes", "Neighborhood_NoRidge", "Neighborhood_NridgHt", "Neighborhood_OldTown", "Neighborhood_SWISU", "Neighborhood_Sawyer", "Neighborhood_Sa

In [12]:
model_data["Garage.Score"] = model_data["Garage.Area"] * model_data["Garage.Qual"]
model_data["ExterQual-s2"] = model_data["Exter.Qual"] ** 2

In [13]:
X = model_data.drop(columns=['SalePrice']).copy()
y = model_data['SalePrice'].copy()

In [14]:
X.values, y.values

(array([[141.0, 31770.0, 1, ..., False, 1056.0, 4],
        [80.0, 11622.0, 0, ..., False, 1460.0, 4],
        [81.0, 14267.0, 1, ..., False, 624.0, 4],
        ...,
        [68.0, 8885.0, 1, ..., False, 968.0, 4],
        [77.0, 10010.0, 0, ..., False, 836.0, 4],
        [74.0, 9627.0, 0, ..., False, 1300.0, 4]], dtype=object),
 array([5.33243846, 5.0211893 , 5.23552845, ..., 5.1172713 , 5.23044892,
        5.27415785]))

In [15]:
from sklearn.model_selection import train_test_split
RANDOM_SEED = 42  # Any number here, really.
Xtrain, Xtest, ytrain, ytest = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=RANDOM_SEED,
)

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.linear_model import LassoCV
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

models = [
    ("Linear Regression", LinearRegression()),
    ("Ridge", Ridge()),
    ("Lasso", Lasso()),
    ("Gradient Boosting", GradientBoostingRegressor(n_estimators=400, max_depth=2, learning_rate=0.1, random_state=42)),
    ("Random Forest", RandomForestRegressor(random_state=42))
]

for model_name, model in models:
    # Criando o pipeline com normalização e o modelo
    pipeline = Pipeline([
        ('scaler', StandardScaler()), 
        ('model', model)
    ])
    
    # Ajustando o modelo
    pipeline.fit(Xtrain, ytrain)
    
    # Fazendo previsões
    ypred = pipeline.predict(Xtest)
    
    # Calculando o RMSE
    RMSE = np.sqrt(mean_squared_error(ytest, ypred))
    
    # Calculando o erro percentual
    error_percent = 100 * (10**RMSE - 1)
    
    # Imprimindo o resultado
    print(f'Model: {model_name} - Average error: {error_percent:.2f}%')


Model: Linear Regression - Average error: 9.86%
Model: Ridge - Average error: 9.86%
Model: Lasso - Average error: 43.52%
Model: Gradient Boosting - Average error: 10.91%
Model: Random Forest - Average error: 12.54%
